# Excel процедуры Filter и Edit, продемонстрированные в Pandas

<a href="https://t.me/init_python"><img src="https://dfedorov.spb.ru/pandas/logo-telegram.png" width="35" height="35" alt="telegram" align="left"></a>

<a href="https://colab.research.google.com/github/dm-fedorov/pandas_basic/blob/master/быстрое%20введение%20в%20pandas/Excel%20функции%20Filter%20и%20Edit%2C%20продемонстрированные%20в%20Pandas.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory" target="_blank"></a>

## Введение

Я слышал от разных людей, что мои предыдущие статьи ([тут](линк1) и [тут](линк2)) об общих задачах Excel в pandas оказались полезными. В этой статье мы продолжим эту традицию, проиллюстрировав различные примеры индексирования pandas с использованием Excel функции `Filter` в качестве модели для понимания процесса.

> Оригинал статьи Криса [здесь](https://pbpython.com/excel-filter-edit.html)

Одна из первых вещей, которую изучает большинство новых пользователей pandas, - это фильтрация данных. Несмотря на то, что я работал с pandas в течение последних нескольких месяцев, недавно я понял, что у подхода к фильтрации pandas есть еще одно преимущество, которое я не использовал в повседневной работе: вы можете фильтровать по заданному набору столбцов, но обновлять другой набор столбцов, используя упрощенный синтаксис pandas. Это похоже на то, что я называю процессом "Фильтрация и редактирование" в Excel.

В этой статье будут рассмотрены некоторые примеры фильтрации `DataFrame` и обновления данных на основе различных критериев. Попутно я объясню еще кое-что об индексировании pandas и о том, как использовать такие методы индексирования, как `.loc` и `.iloc`, для быстрого и легкого обновления подмножества данных на основе простых или сложных критериев.

## Excel: "Фильтрация и редактирование"

Помимо `Pivot Table` (сводной таблицы), одним из самых популярных инструментов в Excel является `Filter`. Этот простой инструмент позволяет быстро фильтровать и сортировать данные по различным числовым, текстовым критериям и критериям форматирования. 

Вот снимок экрана с некоторыми образцами, отфильтрованными по нескольким критериям:

![](https://github.com/dm-fedorov/pandas_basic/blob/master/pic/filter-example.png?raw=True)

Процесс фильтрации интуитивно понятен даже начинающему пользователю Excel. Я также заметил, что люди используют эту функцию для выбора строк данных, а затем обновляют дополнительные столбцы на основе критериев строки. Пример ниже показывает, что я имею в виду:

![](https://github.com/dm-fedorov/pandas_basic/blob/master/pic/commission-example.png?raw=True)

В этом примере я отфильтровал данные по `Account Number` (номеру счета), `SKU` (артикулу) и `Unit Price` (цене за единицу). Затем я вручную добавил столбец `Commission_Rate` и ввел `0.01` в каждую ячейку. Преимущество этого подхода заключается в том, что его легко понять и он может помочь управлять относительно сложными данными без написания длинных формул Excel или использования VBA. Обратной стороной этого подхода является то, что он не воспроизводится, и извне может быть сложно понять, какие критерии использовались для фильтра.

Например, если вы посмотрите на скриншот, нет очевидного способа узнать, что отфильтровано, не глядя на каждый столбец. К счастью, мы можем сделать нечто очень похожее в pandas. 

## Логическое индексирование

Теперь, когда вы понимаете проблему, я хочу подробно рассказать о *логической индексации* (`boolean indexing`) в pandas. Это важная концепция, которую нужно понять, если вы хотите разобраться с [индексированием и выбором данных](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html) в pandas. Эта идея может показаться сложной для начинающего пользователя (и, возможно, слишком простой для опытных), но я думаю, важно потратить некоторое время на ее понимание. Если вы усвоите эту концепцию, то основной процесс работы с данными в pandas упростится.

Pandas поддерживает индексацию (или выбор данных) с помощью меток (labels), целых чисел на основе позиции или списка логических значений (`True`/`False`). Использование списка логических значений для выбора строки называется *логическим индексированием* (`boolean indexing`), и ему будет уделено внимание в остальной части этой статьи.

Я обнаружил, что мой рабочий процесс, как правило, сосредоточен на использовании списков логических значений для выбора данных. Другими словами, когда я создаю `DataFrames`, я стараюсь сохранить в нем индекс по умолчанию. 

> Логическая индексация (`boolean indexing`) - это один из нескольких мощных и полезных способов выбора строк данных в pandas.

Давайте посмотрим на несколько примеров `DataFrames`, чтобы прояснить, что делает логический индекс в pandas.

Во-первых, создадим `DataFrame` из списка Python:

In [ ]:
import pandas as pd
import collections

sales = [('account', ['Jones LLC', 'Alpha Co', 'Blue Inc', 'Mega Corp']),
         ('Total Sales', [150, 200, 75, 300]),
         ('Country', ['US', 'UK', 'US', 'US'])]

# https://github.com/pandas-dev/pandas/issues/21850
df = pd.DataFrame.from_dict(collections.OrderedDict(sales))
df

Обратите внимание, как значения `0-3` автоматически присваиваются строкам. Это индексы, и они не имеют особого значения в этом наборе данных, но полезны для pandas.

Когда мы говорим о логической индексации, то имеем в виду, что можем передать список значений из `True` или `False`, представляющих каждую строку, которую мы хотим посмотреть.

Если хотим посмотреть данные для `Jones LLC`, `Blue Inc` и `Mega Corp`, то список `True` и `False` будет выглядеть следующим образом:

In [ ]:
indices = [True, False, True, True]

Неудивительно, что вы можете передать этот список в `DataFrame`, и он будет отображать только те строки, в которых значение равно `True`:

In [ ]:
df[indices]

Вот визуальное изображение того, что произошло:

![](https://github.com/dm-fedorov/pandas_basic/blob/master/pic/Boolean-Indexing-Example.png?raw=True)

Ручное создание списка индекса работает, но, очевидно, не масштабируется и не очень полезно для чего-либо, кроме тривиального набора данных. К счастью, pandas позволяет очень легко создавать логические индексы, используя простой язык запросов, который должен быть знаком тем, кто использовал Python (или любой другой язык в этом отношении).

Для примера рассмотрим все линии продаж из США:

In [ ]:
df.Country == 'US'

В примере показано, как pandas возьмет вашу традиционную логику Python, применит ее к `DataFrame` и вернет список логических значений. Этот список логических значений затем может быть передан в `DataFrame` для получения соответствующих строк данных.

В реальном коде вы бы не стали выполнять этот двухэтапный процесс. 

Сокращенный вызов выглядит так:

In [ ]:
df[df["Country"] == 'US']

Хотя эта концепция проста, но вы можете написать довольно сложную логику для фильтрации данных, используя возможности Python.

> В этом примере `df[df.Country == 'US']` эквивалентно `df[df["Country"] == 'US']`. Обозначение `.` более чистое, но не будет работать, если в имени столбца присутствуют пробелы.

## Выбор столбцов

Теперь, когда мы выяснили, как выбирать строки данных, как мы можем контролировать, какие столбцы отображать. В приведенном выше примере нет очевидного способа сделать это. Pandas может поддерживать этот вариант, используя два типа индексации на основе местоположения: `.loc` и `.iloc`. Эти функции также позволяют нам выбирать столбцы в дополнение к выбору строк, который мы видели до сих пор.

Существует много недоразумений относительно того, когда использовать `.loc` или `iloc`. Краткое описание различий заключается в следующем:

- `.loc` используется для индексации меток
- `.iloc` используется для целых чисел на основе позиции

Итак, вопрос в том, какой из них использовать? Признаю, что я тоже несколько раз спотыкался на этом. Я обнаружил, что чаще всего использую `.loc`. В основном потому, что мои данные не поддаются осмысленной индексации на основе позиции (другими словами, мне редко нужен `.iloc`), поэтому я придерживаюсь `.loc`.

Честно говоря, у каждого из этих методов есть свое место и они полезны во многих ситуациях. Одна из областей, в частности, связана с иерархической индексацией (`MultiIndex`) `DataFrames`. 

Теперь, когда мы рассмотрели эту тему, давайте покажем, как фильтровать `DataFrame` по значениям в строке и выбирать определенные столбцы для отображения.

Продолжая пример, что, если мы просто хотим показать имена учетных записей (`account`), которые соответствуют нашему индексу? 

Используя `.loc`, это просто:

In [ ]:
df.loc[[True, True, False, True], "account"]

Если вы хотите видеть несколько столбцов, просто передайте список:

In [ ]:
df.loc[[True, True, False, True], ["account", "Country"]]

Настоящая сила - это когда вы создаете более сложные запросы к своим данным. В этом случае давайте покажем все названия аккаунтов (`account`) и страны (`Country`), где продажи `(Total Sales) > 200`:

In [ ]:
df.loc[df["Total Sales"] > 200, ["account", "Country"]]

Этот процесс можно сравнить с фильтром Excel, который мы обсуждали выше. У вас есть дополнительное преимущество: вы также можете ограничить количество извлекаемых столбцов, а не только строк.

## Редактирование столбцов

Все это хорошая основа, но где этот процесс действительно проявляется, так это когда вы используете аналогичный подход для обновления одного или нескольких столбцов на основе выбора строки.

В качестве простого примера давайте добавим к нашим данным столбец `rate` (ставка комиссионного вознаграждения):

In [ ]:
df["rate"] = 0.02
df

Допустим, если вы продали более `100`, ваша ставка составит `5%`. 

Основная задача - установить логический индекс для выбора столбцов, а затем присвоить значение столбцу `rate`:

In [ ]:
df.loc[df["Total Sales"] > 100, ["rate"]] = .05
df

Надеюсь, если вы прочли эту статью, то теперь сможете понять, как работает этот синтаксис. 

Теперь у вас есть основы подхода "Фильтр и редактирование". 

В последнем разделе этот процесс будет более подробно показан в Excel и pandas.


## Собираем все вместе

В последнем примере мы создадим простой калькулятор комиссий, используя следующие правила.

- Все комиссии рассчитываются на уровне транзакции.
- Базовая комиссия со всех продаж составляет `2%`.
- Все рубашки получат комиссию `2.5%`.
- Действует специальная программа, при которой `продажа > 10 ремней` (belts) за одну транзакцию получает комиссию `4%`.
- Существует специальный бонус в размере `250 долларов США плюс комиссия 4.5%` для всех `продаж обуви > 1000 долларов США` за одну транзакцию.

Чтобы сделать это в Excel, используя подход «Фильтр и редактирование»:

- Добавьте столбец комиссии с `2%`.
- Добавьте бонусный столбец `0 долларов`.
- Отфильтруйте рубашки и измените долей на `2.5%`.
- Очистить фильтр.
- Отфильтруйте ремни (`belts`) и `количество (quantity) > 10` и измените значение на `4%`.
- Очистить фильтр.
- Отфильтруйте `обувь > 1000 долларов США` и добавьте комиссию и бонус в размере `4.5%` и `250 долларов США` соответственно.

Я не собираюсь показывать снимки экрана каждого шага, но вот последний фильтр:

![](https://pbpython.com/images/filter-2.png)

Этот подход достаточно прост для манипуляций в Excel, но его нельзя повторить и проверить. Конечно, есть и другие подходы для этого в Excel - например, формулы или VBA. Однако этот подход с фильтром и редактированием является обычным и иллюстрирует логику pandas.

Теперь давайте рассмотрим весь пример в pandas.

Сначала прочтите Excel файл и добавьте столбец со значением по умолчанию `2%`:

In [ ]:
import pandas as pd

df = pd.read_excel("https://github.com/chris1610/pbpython/blob/master/data/sample-sales-reps.xlsx?raw=true")

df["commission"] = .02
df.head()

Следующее правило комиссии: все рубашки получают `2.5%`, а продажи `поясов > 10` получают ставку `4%`:

In [ ]:
df.loc[df["category"] == "Shirt", ["commission"]] = .025
df.loc[(df["category"] == "Belt") & (df["quantity"] >= 10), ["commission"]] = .04
df.head()

Последнее правило комиссии - добавить специальный бонус:

In [ ]:
df["bonus"] = 0
df.loc[(df["category"] == "Shoes") & (df["ext price"] >= 1000 ), ["bonus", "commission"]] = (250, 0.045)

# Показать образец строк, показывающих этот бонус:
df.iloc[3:7]

Для расчета комиссионных:

In [ ]:
# Рассчитайте компенсацию для каждой строки
df["comp"] = df["commission"] * df["ext price"] + df["bonus"]

# Подведите итоги и округлите результаты по торговым представителям
df.groupby(["sales rep"])["comp"].sum().round(2)

## Заключение

Спасибо, что прочитали статью. Я считаю, что одна из самых больших проблем для новых пользователей в изучении того, как использовать pandas, - это выяснить, как использовать свои знания на основе Excel для создания эквивалентного решения на основе pandas. Во многих случаях решение pandas будет более надежным, быстрым, легким для аудита и более мощным. Однако процесс обучения может занять некоторое время. Я надеюсь, что этот пример, показывающий, как решить проблему с помощью инструмента "Фильтр" в Excel, станет полезным руководством для тех, кто только начинает свое pandas путешествие. Удачи!